In [ ]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
from torchsummary import summary

In [ ]:
# Define network base class, this class include network base function
class Net_base(torch.nn.Module):
    def __init__(self):
        super(Net_base, self).__init__()

        # Hyper parameter setting
        self.seed = hash(878787)
        self.batch_size = 50
        self.epochs = 10
        self.lr = 0.0008
        self.momentum = 0.9

        self.no_cuda = False
        self.use_cuda = False
        self.device = None

        # data loader
        self.train_loader = None
        self.test_loader = None

        self.data_loader()
        self.cuda_setting()

    def data_loader(self):
        self.train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./MNIST_data', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
            batch_size=self.batch_size, shuffle=True,num_workers=2)

        self.test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./MNIST_data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
            batch_size=self.batch_size, shuffle=True,num_workers=2)

        print("data loading finish")

    def cuda_setting(self):
        # cuda setting
        self.use_cuda = not self.no_cuda and torch.cuda.is_available()
        torch.manual_seed(self.seed)
        if self.use_cuda:
            torch.cuda.manual_seed(self.seed)
            self.device = torch.device('cuda')
            self.to(self.device)

        print("device is: {}".format(self.device))

    def train_step(self, epoch):
    # training function
        self.train()
        for batch_idx, (data, target) in enumerate(self.train_loader):
            if self.use_cuda:
                data, target = data.to(self.device), target.to(self.device)

            self.optimizer.zero_grad()
            output = self.__call__(data) # model(data)
            loss = self.loss_fn(output, target)
            loss.backward()
            self.optimizer.step()
            if (batch_idx+1) % 300 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, (batch_idx+1) * len(data), len(self.train_loader.dataset),
                        100. * (batch_idx+1) / len(self.train_loader), loss.item()))

    # testing function
    def test_step(self, epoch):
        self.eval()
        test_loss = 0
        correct = 0
        for batch_idx, (data, target) in enumerate(self.test_loader):
            if self.use_cuda:
                data, target = data.to(self.device), target.to(self.device)
            with torch.no_grad():
                output = self.__call__(data)
                test_loss += self.loss_fn(output, target).item()
                pred = output.data.max(1)[1] # get the index of the max log-probability
                correct += pred.eq(target.data).cpu().sum()

        test_loss /= len(self.test_loader) # loss function already averages over batch size
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
            test_loss, correct, len(self.test_loader.dataset),
            100. * correct / len(self.test_loader.dataset)))

    def training_model(self, epochs=0, save_weight=False):
        # run and save model
        if epochs == 0:
            epochs = self.epochs
        for epoch in range(1, epochs + 1):
            self.train_step(epoch)
            self.test_step(epoch)
            if save_weight:
                savefilename = 'LeNet_'+str(epoch)+'.tar'
                torch.save({
                        'epoch': epoch,
                        'state_dict': self.state_dict(),
                    }, savefilename)

In [ ]:
# LeNet class, inheritance to net base
class LeNet(Net_base):
    def __init__(self):
        super(LeNet, self).__init__()

        # define layer
        self.conv1 = torch.nn.Conv2d(1, 6, kernel_size=(5,5),stride=1, padding=0)
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=(5,5),stride=1, padding=0)
        self.fc1 = torch.nn.Linear(16*4*4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

        # define loss function
        self.loss_fn = torch.nn.CrossEntropyLoss()

        # define optimizer
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        
    # use print function can show model architecture
    def __repr__(self):
        if self.use_cuda:
            summary(self.cuda(), (1, 28, 28))
        else:
            summary(self, (1, 28, 28))
        return ""

    def forward(self, x):
        out = torch.nn.functional.relu(self.conv1(x))
        out = torch.nn.functional.max_pool2d(out, 2)
        out = torch.nn.functional.relu(self.conv2(out))
        out = torch.nn.functional.max_pool2d(out, 2)
        out = out.view(out.size(0), -1) #flatten
        out = torch.nn.functional.relu(self.fc1(out))
        out = torch.nn.functional.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
model = LeNet()
print(model)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw

data loading finish
device is: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 16, 8, 8]           2,416
            Linear-3                  [-1, 120]          30,840
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.21
----------------------------------------------------------------



In [ ]:
model.training_model()

Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.278294
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.049569
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.045579
Train Epoch: 1 [60000/60000 (100%)]	Loss: 0.015255

Test set: Average loss: 0.0872, Accuracy: 9717/10000 (97.2%)

Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.019389
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.029503
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.026528
Train Epoch: 2 [60000/60000 (100%)]	Loss: 0.079579

Test set: Average loss: 0.0510, Accuracy: 9840/10000 (98.4%)

Train Epoch: 3 [15000/60000 (25%)]	Loss: 0.013784
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.050973
Train Epoch: 3 [45000/60000 (75%)]	Loss: 0.025955
Train Epoch: 3 [60000/60000 (100%)]	Loss: 0.011792

Test set: Average loss: 0.0470, Accuracy: 9848/10000 (98.5%)

Train Epoch: 4 [15000/60000 (25%)]	Loss: 0.027468
Train Epoch: 4 [30000/60000 (50%)]	Loss: 0.011756
Train Epoch: 4 [45000/60000 (75%)]	Loss: 0.002488
Train Epoch: 4 [60000/60000 (100%)]	Loss: 0.003399

Test s

In [ ]:
import torch.onnx
def Convert_ONNX(): 
    # set the model to inference mode 
    model.eval()
    # Let's create a dummy input tensor  
    dummy_input = torch.randn(1, 1, 28, 28, requires_grad=True).cuda()

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "mnist.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput']) # the model's output names
    print('Model has been converted to ONNX') 

In [ ]:
Convert_ONNX()

Model has been converted to ONNX
